In [1]:
from tomoSegmentPipeline.utils.common import read_array, write_array
from tomoSegmentPipeline.utils import setup

from cryoS2Sdrop.dataloader import singleCET_dataset
from cryoS2Sdrop.model import Denoising_UNet
from cryoS2Sdrop.losses import self2self_L2Loss
from cryoS2Sdrop.trainer import denoisingTrainer

import os
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from torchsummary import summary
from pytorch_lightning import Trainer
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import LearningRateMonitor

PARENT_PATH = setup.PARENT_PATH
ISONET_PATH = os.path.join(PARENT_PATH, 'data/isoNet/')

%matplotlib inline
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [2]:
cet_path = os.path.join(PARENT_PATH, 'data/raw_cryo-ET/tomo02.mrc') 
p=0.3 # dropout probability
subtomo_length = 96
n_bernoulli_samples = 50
n_features = 48
tensorboard_logdir = os.path.join(PARENT_PATH, 'data/S2SDenoising/tryout_model_logs')
batch_size = 8
epochs = 2
num_gpus = 2
lr = 5e-4

s2s_trainer = denoisingTrainer(cet_path, subtomo_length, lr, n_bernoulli_samples, n_features, p, tensorboard_logdir)

In [3]:
s2s_trainer.train(batch_size, epochs, num_gpus, accelerator = 'gpu', strategy = 'dp')

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Size of dataset: 500, Steps per epoch: 31. 



LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

   | Name      | Type             | Params
------------------------------------------------
0  | loss_fn   | self2self_L2Loss | 0     
1  | EB1       | PartialConv3d    | 64.8 K
2  | EB2       | Sequential       | 62.3 K
3  | EB3       | Sequential       | 62.3 K
4  | EB4       | Sequential       | 62.3 K
5  | EB5       | Sequential       | 62.3 K
6  | EB6       | Sequential       | 62.3 K
7  | EB_bottom | Sequential       | 62.3 K
8  | up65      | Upsample         | 0     
9  | up54      | Upsample         | 0     
10 | up43      | Upsample         | 0     
11 | up32      | Upsample         | 0     
12 | up21      | Upsample         | 0     
13 | DB5       | Sequential       | 497 K 
14 | DB4       | Sequential       | 622 K 
15 | DB3       | Sequential       | 622 K 
16 | DB2       | Sequential       | 622 K 
17 | DB1       | Sequential       | 350 K 
------------------------------------------------
3.2 M     Trainable params
0         Non

0
0
1
1
2
2
3
3
4
4
5
5
torch.Size([4, 50, 96, 96, 96])
torch.Size([4, 50, 96, 96, 96])
6
6
7
7
8
8
9
9
10
10
11
11
12
12
13
13
14
14
1515

1616

1717

18
18
19
19
20
20
21
21
22
22
23
23
24
24
25
25
26
26
27
27
28
28
29
29
30
30
31
31
32
32
33
33
34
34
35
35
36
36
37
37
38
38
39
39
40
40
41
41
42
42
43
43
44
44
45
45
46
46
47
47
48
48
49
49
50
50
51
51
52
52
53
53
54
54
55
55
56
56
57
57
58
58
59
59
60
60
61
61
62
62


Metric hp/train_loss improved. New best score: 33978252.000


AttributeError: 'NoneType' object has no attribute 'squeeze'

# Loading model

In [10]:
ckpt_file = os.path.join(PARENT_PATH, 'data/S2SDenoising/tryout_model_logs/version_0/checkpoints/epoch=1-step=126.ckpt')

model = Denoising_UNet.load_from_checkpoint(ckpt_file)